## E-commerce with RFM and Clustering

## Business Problem
   **What is the business problem?**

    Select the most valuable customers to join the "Loyals" loyalty program.

    Answering business questions to the marketing team.

## Output
**What will I deliver? / Where does the business team want to see it?**

List in xls / send by email. It must contain customers who will join Loyals (loyalty program).
Format:
client_id	is_loyal

    │1	│yes
    │2	│no
    
**PDF report answering business questions / send by email and present to the marketing team:**

    Who are the people eligible to participate in the Loyals program?
    How many customers will be part of the group?
    What are the main characteristics of these customers?
    What is the percentage of revenue contribution from Loyals?
    What is the expected revenue for this group for the coming months?
    What are the conditions for a person to be eligible for Loyals?
    What are the conditions for a person to be removed from Loyals?
    What is the guarantee that the Loyals program is better than the rest of the base?
    What actions can the marketing team take to increase revenue?

### Input

Data sources: Dataset "Ecommerce.csv", containing e-commerce sales for the period of one year.

Tools: Python 3.8.12, Jupyter Notebook, Git, Github.

Process
Type of problem: Separation of customers by groups.

Main methods: clustering.

**Business Questions:**

    Who are the people eligible to participate in the Loyals program?
    
    What does it mean to be eligible? / What are higher "value" customers? (depending on business area)
    Invoicing:
    
   - High average ticket
       High LTV (sum of customer revenue)
       
       Low recency (time since last purchase)
       
       High basket size (qty products purchased per purchase)
       
       Low churn probability (would use the output of a model)
       
       High LTV Prediction (would use output from a model)
       
       High propensity to buy (would use the output of a model)
       
              
   - Cost:
        Low return rate
        
   - Shopping experience:
   
        High average of the evaluations

## Solution Benchmarking
Desk Research
Reading articles on customer segmentation on the internet to understand what the market is doing.

Identify from market solutions, with the business team, what we can do as an MVP.

RFM segmentation model.

# Imports

In [55]:
import re
import sweetviz as sv
import inflection
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn import preprocessing as pp
from sklearn.cluster import KMeans
from sklearn import metrics as m
import umap.umap_   as umap
from sklearn             import decomposition as dd
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


warnings.filterwarnings( 'ignore' )

## Helper Functions

In [2]:
# Função para formatar notação científica
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Tamanho dos Graficos
sns.set(style = "darkgrid")
sns.set(rc={"figure.figsize":(15, 10)})
#pd.options.display.float_format = "{:,.2f}".format


## Load Data

In [3]:
df_raw = pd.read_csv('data/ecommerce.csv',header= 0, encoding= 'unicode_escape')

In [4]:
df_raw =df_raw.drop(['Unnamed: 8'], axis=1)

In [5]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.3900,17850.0000,United Kingdom


# Data Description

In [6]:
df1 = df_raw.copy()

## Rename Columns

In [7]:
df1.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [9]:
#Função para renomear colunas
cols_old = list(df1.columns)
snakecase = lambda x: inflection.underscore(x)

cols_new = list( map( snakecase, cols_old) )

#Rename
df1.columns = cols_new

## Data Dimensions

In [10]:
print(f'Number of Columns: {df1.shape[1]}\n')
print(f'Number of Rows: {df1.shape[0]}')



Number of Columns: 8

Number of Rows: 541909


## Data Types

In [11]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## Check NA

In [12]:
df1.isna().sum()/len(df1)

invoice_no     0.0000
stock_code     0.0000
description    0.0027
quantity       0.0000
invoice_date   0.0000
unit_price     0.0000
customer_id    0.2493
country        0.0000
dtype: float64

**Há um grande numero de valores NA na coluna customer ID.**
- Não faria sentido preencher com média ou moda, vamos excluí-los.

### Drop NA

In [13]:
df1 = df1.dropna(subset=['customer_id'])

## Change dtypes

In [14]:
# invoicedate
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

#Customer_id
df1['customer_id'] = df1['customer_id'].astype('int64')

In [15]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
customer_id              int64
country                 object
dtype: object

## Descriptive Statistics

In [16]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

### Numerical Attributes

In [17]:
#central tendency - mean, median, 
ct1 = pd.DataFrame(num_attributes.apply(np.mean) ).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

#Dispersion - std, 
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(np.min)).T
d3 = pd.DataFrame(num_attributes.apply(np.max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew()) ).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis()) ).T

#Merge
n = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
n.columns = ['attributes', 'min', 'max', 'range','mean', 'median', 'std','skew','kurtosis']
n

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,quantity,-80995.0000,80995.0000,161990.0000,12.0613,5.0000,248.6931,0.1827,94317.5637
1,unit_price,0.0000,38970.0000,38970.0000,3.4605,1.9500,69.3151,452.2190,246924.5430
2,customer_id,12346.0000,18287.0000,5941.0000,15287.6906,15152.0000,1713.5982,0.0298,-1.1800


 ### Categorical Attributes

In [18]:
# check unique values of categorical features
cat_attributes.apply( lambda x: x.unique().shape[0])

invoice_no     22190
stock_code      3684
description     3896
country           37
dtype: int64

### Country

In [19]:
#purchase to countries

df1['country'].value_counts(normalize=True)

United Kingdom         0.8895
Germany                0.0233
France                 0.0209
EIRE                   0.0184
Spain                  0.0062
Netherlands            0.0058
Belgium                0.0051
Switzerland            0.0046
Portugal               0.0036
Australia              0.0031
Norway                 0.0027
Italy                  0.0020
Channel Islands        0.0019
Finland                0.0017
Cyprus                 0.0015
Sweden                 0.0011
Austria                0.0010
Denmark                0.0010
Japan                  0.0009
Poland                 0.0008
USA                    0.0007
Israel                 0.0006
Unspecified            0.0006
Singapore              0.0006
Iceland                0.0004
Canada                 0.0004
Greece                 0.0004
Malta                  0.0003
United Arab Emirates   0.0002
European Community     0.0001
RSA                    0.0001
Lebanon                0.0001
Lithuania              0.0001
Brazil    

In [20]:
df1[['customer_id', 'country']].drop_duplicates().groupby('country').count().reset_index().sort_values('customer_id', ascending=False).head()

,country,customer_id
35,United Kingdom,3950
14,Germany,95
13,France,87
30,Spain,31
3,Belgium,25


Há mais clientes no Reino Unido (United Kingdom)com aproximadamente 4.000, seguido da Alemanha (Germany) e França (France) com 95 e 87, respectivamente.

### Invoice_no

In [21]:
# problema: A coluna invoice é composta por numeros e letras

df_letter_invoices = df1.loc[df1['invoice_no'].apply( lambda x: bool( re.search( '[^0-9]+', x ) ) ), :]

#Verificar se todos os invoices são negativos
print(f'total de invoices com letras: {len(df_letter_invoices)}')

#Verificar se todos os invoices são negativos
print(f'total de invoices com valores negativos: {len(df_letter_invoices[ df_letter_invoices["quantity"] < 0 ])}')

total de invoices com letras: 8905
total de invoices com valores negativos: 8905


# Filtering Variable

In [22]:
df2 =df1.copy()

In [23]:
#=====================Numerical Attributes=================================
#Quantity
df2 = df2.loc[df2['unit_price']>= 0.04]


#=====================Caterical Attributes=================================
df2=  df2[~df2['stock_code'].isin( ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]

#Description
df2 = df2.drop(columns='description', axis=1)


#map
df2 = df2[~df2['country'].isin(['European Community', 'Unspecified'])]

#Quantity
df_return = df2.loc[df1['quantity']<0, :]
df2_purchase = df2.loc[df1['quantity']>= 0, :]


#Remove Bad User 
df2 = df2[~df2['customer_id'].isin( [16446] )]


# Feature Engeneering

In [24]:
df3 = df2.copy()

In [25]:
# Criar colunas Recency, Frequency, Monetary
# Data reference
df_ref = df3.drop(['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price', 
                   'country'], axis=1).drop_duplicates(ignore_index=True)

### Gross Revenue

In [26]:
#Gross Revenue (Faturamento) - Quantity * price
df2_purchase[ 'gross_revenue'] = df2_purchase['quantity'] * df2_purchase['unit_price']

#Monetary
df_monetary = df2_purchase[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()

df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')

### Recency - days since the last purchase

In [27]:
# Recency - Last day purchase
df_recency = df2_purchase.loc[:, ['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
df_recency['recency_days'] = ( df2['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id       0
gross_revenue    27
recency_days     27
dtype: int64

### Quantity of Purchases by Customer


In [28]:
# Qty of invoice no per customer
df_invoice_no = df2_purchase[['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index().rename(columns = {'invoice_no': 'qty_invoice_no'})

df_ref = pd.merge(df_ref, df_invoice_no, on = 'customer_id', how = 'left')
df_ref.isna().sum()

customer_id        0
gross_revenue     27
recency_days      27
qty_invoice_no    27
dtype: int64

### Quantity of products (different stock codes by customer)

In [29]:
#Products number
df_freq = (df2_purchase.loc[:,['customer_id', 'stock_code']].groupby('customer_id')
                                                            .count()
                                                            .reset_index()
                                                            .rename(columns={'stock_code':'qtde_products'}))
           
df_ref =pd.merge(df_ref, df_freq, on='customer_id', how='left')
df_ref.isna().sum()

customer_id        0
gross_revenue     27
recency_days      27
qty_invoice_no    27
qtde_products     27
dtype: int64

### Quantity of Items Purchased by Customer

In [30]:
# Quantity of items purchased by customer
df_freq = (df2_purchase.loc[:, ['customer_id', 'quantity']].groupby( 'customer_id' ).sum()
                                                           .reset_index()
                                                           .rename( columns={'quantity': 'qty_items'} ) )
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )
df_ref.isna().sum()

customer_id        0
gross_revenue     27
recency_days      27
qty_invoice_no    27
qtde_products     27
qty_items         27
dtype: int64

### Frequency

In [31]:
df_aux = (df2_purchase[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                           .groupby('customer_id')
                                                           .agg(max_ =('invoice_date', 'max'),
                                                                min_ =('invoice_date', 'min'),
                                                                days_ =('invoice_date', lambda x:(( x.max() - x.min()).days)+1),
                                                                buy_ =('invoice_date', 'count'))).reset_index()
#Frequency
df_aux['frequency'] =df_aux[['buy_', 'days_']].apply( lambda x: x['buy_']/ x['days_'] if x['days_'] != 0 else 0, axis=1)

#Merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], how='left', on='customer_id')

### Average Ticket Value

In [32]:
# Average Ticket Value
df_avg_ticket = df2_purchase.loc[:, ['customer_id', 'gross_revenue']].groupby( 'customer_id' ).mean().reset_index().rename( columns={'gross_revenue':'avg_ticket'} )
df_ref = pd.merge( df_ref, df_avg_ticket, on='customer_id', how='left')
df_ref.isna().sum()

customer_id        0
gross_revenue     27
recency_days      27
qty_invoice_no    27
qtde_products     27
qty_items         27
frequency         27
avg_ticket        27
dtype: int64

### Number of Returns

In [33]:
#Number of Returns
df_returns = df_return[['customer_id', 'quantity']].groupby( 'customer_id' ).sum().reset_index().rename( columns={'quantity':'qty_returns'} )
df_returns['qty_returns'] = df_returns['qty_returns'] * -1

df_ref = pd.merge( df_ref, df_returns, how='left', on='customer_id' )
df_ref.loc[df_ref['qty_returns'].isna(), 'qty_returns'] = 0 #customers with 0 returned items

df_ref.isna().sum()

customer_id        0
gross_revenue     27
recency_days      27
qty_invoice_no    27
qtde_products     27
qty_items         27
frequency         27
avg_ticket        27
qty_returns        0
dtype: int64

### Basket Size

In [34]:
#Basket Size - the number of products sold in a single purchase ( Quantity )
# Invoice No = Purchase = purchase
# Stock Code = Produto Único = Product
# Quantity = Item = Item

df_aux = ( df2_purchase.loc[:, ['customer_id', 'invoice_no', 'quantity']].groupby( 'customer_id' )
                                                                            .agg( n_purchase=( 'invoice_no', 'nunique'),
                                                                                   n_products=( 'quantity', 'sum' ) )
                                                                            .reset_index() )

# calculation
df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'avg_basket_size']], how='left', on='customer_id' )
df_ref.isna().sum()

customer_id         0
gross_revenue      27
recency_days       27
qty_invoice_no     27
qtde_products      27
qty_items          27
frequency          27
avg_ticket         27
qty_returns         0
avg_basket_size    27
dtype: int64

### Unique Basket Size

In [35]:
#Unique Basket Size - Quantidade de produtos distintos por compra

df_aux = ( df2_purchase.loc[:, ['customer_id', 'invoice_no', 'stock_code']].groupby( 'customer_id' )
                                                                            .agg( n_purchase=( 'invoice_no', 'nunique'),
                                                                                   n_products=( 'stock_code', 'nunique' ) )
                                                                            .reset_index() )

# calculation
df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

# merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'avg_unique_basket_size']], how='left', on='customer_id' )
df_ref.isna().sum()

customer_id                0
gross_revenue             27
recency_days              27
qty_invoice_no            27
qtde_products             27
qty_items                 27
frequency                 27
avg_ticket                27
qty_returns                0
avg_basket_size           27
avg_unique_basket_size    27
dtype: int64

# EDA

In [36]:
df4 = df_ref.copy()

In [37]:
df4.isna().sum()

customer_id                0
gross_revenue             27
recency_days              27
qty_invoice_no            27
qtde_products             27
qty_items                 27
frequency                 27
avg_ticket                27
qty_returns                0
avg_basket_size           27
avg_unique_basket_size    27
dtype: int64

**O cluster deve ser coesos e separados**
**Métricas que indicam variabilidade:** 

    - Min, Max, Range (Dispersão)
    
    - Media e Mediana
    
    - Desvio padrão e Variancia
    
    - Distribuição
    
    - Coeficiente de Variação
    
    - Olhar com atenção pros outliers

## Análise Univariada
Usar o Sweetviz

In [38]:
#generate report with Pandas Profiling Report
#my_report2 = sv.analyze(df4)
#my_report2.show_html() 

- AO analisar o relatório do sweetviz observei que tem um valor muito alto. Vamos dar um zoom nesse id e ver do que se trata

### Quantity Items

In [39]:
df4.loc[df4['qty_items']==196844]

,customer_id,gross_revenue,recency_days,qty_invoice_no,qtde_products,qty_items,frequency,avg_ticket,qty_returns,avg_basket_size,avg_unique_basket_size
908,14646,279138.0200,1.0000,72.0000,2060.0000,196844.0000,0.2034,135.5039,288.0000,2733.9444,9.7083


Esse cliente fez a compra ha 1 dia, comprou 2060 produtos m tem uma frequencia de 20

In [40]:
df3[df3['customer_id']==14646].head(20)

,invoice_no,stock_code,quantity,invoice_date,unit_price,customer_id,country
37952,539491,21981,12,2016-12-18,0.2900,14646,Netherlands
37953,539491,21986,12,2016-12-18,0.2900,14646,Netherlands
37954,539491,22720,2,2016-12-18,4.9500,14646,Netherlands
37955,539491,21931,1,2016-12-18,1.9500,14646,Netherlands
37956,539491,22613,2,2016-12-18,0.8500,14646,Netherlands
37957,539491,20751,1,2016-12-18,2.1000,14646,Netherlands
37958,539491,21246,2,2016-12-18,4.9500,14646,Netherlands
37959,539491,22960,1,2016-12-18,4.2500,14646,Netherlands
37960,539491,22355,2,2016-12-18,0.8500,14646,Netherlands
37961,539491,21123,2,2016-12-18,1.2500,14646,Netherlands


**- Ao que aparece é um cliente recorrente que compra grandes quantidades em pequenos valores**

### AVG Ticket

O AVG tciket também demosntrou um valor discrepante.
Percebe-se que é o mesmo cliente anterior, sendo assim, a decisão é de excluir esse id, pois ele gera um outlier e distorce as informações dos dados.

In [41]:
#df4[df4['avg_ticket']==56157.5]

### Frequency

In [42]:
df4[df4['frequency']==17]

,customer_id,gross_revenue,recency_days,qty_invoice_no,qtde_products,qty_items,frequency,avg_ticket,qty_returns,avg_basket_size,avg_unique_basket_size
0,17850,5391.2100,372.0000,34.0000,297.0000,1733.0000,17.0000,18.1522,40.0000,50.9706,0.6176


In [43]:
df3[df3['customer_id']==17850].head(20)

,invoice_no,stock_code,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,6,2016-11-29,2.5500,17850,United Kingdom
1,536365,71053,6,2016-11-29,3.3900,17850,United Kingdom
2,536365,84406B,8,2016-11-29,2.7500,17850,United Kingdom
3,536365,84029G,6,2016-11-29,3.3900,17850,United Kingdom
4,536365,84029E,6,2016-11-29,3.3900,17850,United Kingdom
5,536365,22752,2,2016-11-29,7.6500,17850,United Kingdom
6,536365,21730,6,2016-11-29,4.2500,17850,United Kingdom
7,536366,22633,6,2016-11-29,1.8500,17850,United Kingdom
8,536366,22632,6,2016-11-29,1.8500,17850,United Kingdom
47,536372,22632,6,2016-11-29,1.8500,17850,United Kingdom


O gráfico mostra que a distribuição é afetada pelos valores extremos. Portanto, a maioria dos clientes gastou menos de 1,00 durante o periodo em análise.

### AVG Basket size

- No gráfico é possível notar que a maioria das compras foram realizadas nos últimos 100 dias
- Mas há clientes que não fazem compras ha 373 dias

In [44]:
#df4[df4['avg_basket_size']==40498.5]

## Bivariate Analysis

- analyze a feature in relation to all others

In [45]:
#cols = ['customer_id']
df4 = df3.copy()

In [46]:
#plt.figure( figsize = (25, 12))
#sns.pairplot(df4);

**Notes**

    - Frequency has low variance
    - Avg ticket has low variance

# Data Preparation

In [58]:
df43 = df4.drop(columns=['customer_id'], axis=1).copy()
df43.head()

,invoice_no,stock_code,quantity,invoice_date,unit_price,country
0,536365,85123A,6,2016-11-29,2.5500,United Kingdom
1,536365,71053,6,2016-11-29,3.3900,United Kingdom
2,536365,84406B,8,2016-11-29,2.7500,United Kingdom
3,536365,84029G,6,2016-11-29,3.3900,United Kingdom
4,536365,84029E,6,2016-11-29,3.3900,United Kingdom


In [51]:
#df43.columns

In [59]:
df43 = df4.dropna()

In [63]:
df4.columns

Index(['invoice_no', 'stock_code', 'quantity', 'invoice_date', 'unit_price',
       'customer_id', 'country'],
      dtype='object')

In [61]:
mm = pp.MinMaxScaler()

#
df43['gross_revenue']          = mm.fit_transform(df43[['gross_revenue']])
df43['recency_days']           = mm.fit_transform(df43[['recency_days']])
df43['qty_invoice_no']         = mm.fit_transform(df43[['qty_invoice_no']])
df43['qty_items']              = mm.fit_transform(df43[['qty_items']])
df43['qty_products']           = mm.fit_transform(df43[['qty_products']])
df43['avg_ticket']             = mm.fit_transform(df43[['avg_ticket']])
df43['avg_recency_days']       = mm.fit_transform(df43[['avg_recency_days']])
df43['frequency']              = mm.fit_transform(df43[['frequency']])
df43['qtde_returns']           = mm.fit_transform(df43[['qtde_returns']])
df43['avg_basket_size']        = mm.fit_transform(df43[['avg_basket_size']])
df43['avg_unique_basket_size'] = mm.fit_transform(df43[['avg_unique_basket_size']])

X = df43.copy()

KeyError: "None of [Index(['gross_revenue'], dtype='object')] are in the [columns]"

# Estudo do Espaço

In [ ]:
#cols_selected = ['customer_id', 'gross_revenue','recency_days', 'qtde_products', 'frequency', 'qty_returns']
#X = df43[cols_selected].copy()

Existem diversas técnicas de redução de dimensionalidade, incluindo métodos lineares (como PCA e LDA) e não lineares (como t-SNE e UMAP). Essas técnicas podem ser utilizadas para encontrar combinações de variáveis que preservem a maior quantidade possível de informações relevantes, gerando um conjunto de dados reduzido com menos variáveis.

## PCA

In [ ]:
#define number of features on space
pca = dd.PCA (n_components=X.shape[1])

principal_components = pca.fit_transform( X )

#plot explianed variables (variance of the main components)
#x axis
features = range(pca.n_components_)
#y axis
plt.bar(features, pca.explained_variance_ratio_, color='black')

#pca_component
df_pca = pd.DataFrame(principal_components)

In [ ]:
df_pca.head()

In [ ]:
sns.scatterplot(x=0, y=1, data=df_pca);

- There is no clear division, let's keep going.

## UMAP

O UMAP é uma técnica de redução de dimensionalidade que pode ser utilizada para visualização de dados em um espaço bidimensional ou tridimensional. Ele não é uma técnica de clusterização em si, mas pode ser utilizado em conjunto com outras técnicas, como o K-means, para agrupar os dados em clusters.

In [ ]:
reducer = umap.UMAP( random_state=42 )
embedding = reducer.fit_transform( X )
df_pca = pd.DataFrame()
df_pca['embedding_x'] = embedding[:, 0]
df_pca['embedding_y'] = embedding[:, 1]
sns.scatterplot( x='embedding_x',
                 y='embedding_y',
                 data=df_pca )

## t-SNE

In [ ]:
reducer = TSNE( n_components=2, n_jobs=-1, random_state=42 ) #can user different n_components, try it!
embedding = reducer.fit_transform( X )
df_tsne = pd.DataFrame()
df_tsne['embedding_x'] = embedding[:, 0]
df_tsne['embedding_y'] = embedding[:, 1]
sns.scatterplot( x='embedding_x', y='embedding_y', data=df_tsne );

### Tree based embeding

**Tree-based Embedding** é uma técnica de clusterização que utiliza árvores de decisão para gerar representações numéricas dos dados, que são utilizadas para agrupar os dados em clusters com base em sua similaridade.

A principal vantagem dessa técnica é permitir trabalhar com dados de alta dimensionalidade, sem a necessidade de reduzir a dimensionalidade dos dados, o que pode ser útil em casos em que a redução da dimensionalidade pode levar à perda de informações importantes.

In [ ]:
#training dataset
X= df43.drop('gross_revenue', axis=1)
 
y = df43['gross_revenue']

#model definition
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Model Training
rf_model.fit(X, y)
#Leaf

#Dataframe
df_leaf = pd.DataFrame(rf_model.apply(X))
# Reduzer dimensionality

In [ ]:
df_leaf.shape

### UMAP

In [ ]:
reducer = umap.UMAP()
# Reduzer dimensionality
reducer = umap.UMAP( random_state=42 )
embedding = reducer.fit_transform( df_leaf )

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

In [ ]:
# Plot UMAP
sns.scatterplot(x='embedding_x',
               y='embedding_y',
               data= df_tree)

# Data Preparation

In [ ]:
df5 = df4.copy()

In [ ]:
df5 =df5.dropna()

In [ ]:
mm = MinMaxScaler()

df5['gross_revenue']          =mm.fit_transform( df5[['gross_revenue']])
df5['recency_days']           =mm.fit_transform( df5[['recency_days']])
df5['qty_invoice_no']         =mm.fit_transform( df5[['qty_invoice_no']])
df5['qtde_products']          =mm.fit_transform( df5[['qtde_products']])
df5['qty_items']              =mm.fit_transform( df5[['qty_items']])
df5['frequency']              =mm.fit_transform( df5[['frequency']])
df5['avg_ticket']             =mm.fit_transform( df5[['avg_ticket']])
df5['qty_returns']            =mm.fit_transform( df5[['qty_returns']])
df5['avg_basket_size']        =mm.fit_transform( df5[['avg_basket_size']])
df5['avg_unique_basket_size'] =mm.fit_transform( df5[['avg_unique_basket_size']])

# Feature Selection

In [ ]:
df6 = df5.drop('customer_id', axis=1)

## Hyper Parameter Fine-Tuning

In [ ]:
X=df6.copy()

In [ ]:
X.head()

# Model Training

In [ ]:
clusters=[2,3,4,5,6,7]

## K-Means

K-Means is heavily influenced by the presence of outliers as they increase significantly the loss function that the algorithm tries to minimize. This loss function is the squared sum of the distances of each datapoint to the centroid, so, if the outlier is far enough, the centroid will be incorrectly situated. 

In [ ]:
#set number of clusters
clusters_in = np.arange(2,13,1)



kmeans_list = []
for k in clusters_in:
    #model definition
    kmeans_model = KMeans( n_clusters=k, n_init=100, random_state=42 )#P.S: use always this 2 parameters: n_init=300, random_state=42, or when running again, values of clustering will change.

    #model training
    kmeans_model.fit(X)

    #Model predict
    kmeans_labels = kmeans_model.predict(X)
    #labels: array([1, 5, 5, ..., 5, 3, 5], dtype=int32)
        
   #model perfomance (SS)
    kmeans_ss = m.silhouette_score(X, kmeans_labels, metric='euclidean')
    kmeans_list.append(kmeans_ss)

In [ ]:
#Silhouette Score 
#n_estimators= 100, 5 features: ['gross_revenue','recency_days','unique_products','daily_purchase_rate','total_prod_returned']
plt.plot(clusters_in, kmeans_list, linestyle='--', marker='o', color='b')
plt.xlabel('K');
plt.ylabel('Silhouette Score');
plt.title('Silhouette Score x K');
plt.show


## Gaussian Mixture Model

In [62]:
k=8
#Model definition
gmm_model = GaussianMixture(n_components=k, n_init=300, random_state=42)


#Model training
gmm_model.fit( X )

#Clustering
labels = gmm_model.predict(X)

NameError: name 'X' is not defined

## Cluster Validation

In [ ]:
# SS (Silhouette Score)
print( 'SS value: {}'.format( m.silhouette_score( X, labels, metric='euclidean' ) ) )